![](https://img-blog.csdnimg.cn/20210626105321101.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzM3NTQxMDk3,size_16,color_FFFFFF,t_70#pic_center)



所以实现的思路可以从这三点开始。
1. Linear Projection of Flattened Patches(Embedding层)
2. Transformer Encoder(图右侧有给出更加详细的结构)
3. MLP Head（最终用于分类的层结构）


In [9]:
import torch
x = torch.tensor([[1],[2]])
x.size()

torch.Size([2, 1])

Embedding层实现的方式：
原理：对于图像数据而言，其数据格式为[H, W, C]是三维矩阵明显不是Transformer想要的。所以需要先通过一个Embedding层来对数据做个变换。如下图所示，首先将一张图片按给定大小分成一堆Patches。以ViT-B/16为例，将输入图片(224x224)按照16x16大小的Patch进行划分，划分后会得到( 224 / 16 ) 2 = 196。接着通过线性映射将每个Patch映射到一维向量中，以ViT-B/16为例，每个Patche数据shape为[16, 16, 3]通过映射得到一个长度为768的向量（后面都直接称为token）。[16, 16, 3] -> [768]

代码实现：在代码实现中，直接通过一个卷积层来实现。 以ViT-B/16为例，直接使用一个卷积核大小为16x16，步距为16，卷积核个数为768的卷积来实现。通过卷积[224, 224, 3] -> [14, 14, 768]，然后把H以及W两个维度展平即可[14, 14, 768] -> [196, 768]，此时正好变成了一个二维矩阵，正是Transformer想要的。

然后要插上Class_token和Position Embedding.
在原论文中，作者说参考BERT，在刚刚得到的一堆tokens中插入一个专门用于分类的[class]token，**这个[class]token是一个可训练的参数**，数据格式和其他token一样都是一个向量，以ViT-B/16为例，就是一个长度为768的向量，与之前从图片中生成的tokens拼接在一起，Cat([1, 768], [196, 768]) -> [197, 768]。然后关于Position Embedding就是之前Transformer中讲到的Positional Encoding，这里的Position Embedding采用的是一个可训练的参数（1D Pos. Emb.），是直接叠加在tokens上的（add），所以shape要一样。以ViT-B/16为例，刚刚拼接[class]token后shape是[197, 768]，那么这里的Position Embedding的shape也是[197, 768]。

In [ ]:
import torch.nn as nn
class MyEmbedding(nn.Module):
    def __init__(self,patch_size,kernel_size,kernel_stride,in_channels,out_channels):
        super(MyEmbedding, self).__init__()
        self.patch_size = patch_size
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size,kernel_stride)

    def forward(self,x):
        # x是整个图片
        x  = self.conv1(x) # 获取到Patches的信息。     [224, 224, 3] -> [14, 14, 768]
        shape = x.size()
        x = x.reshape(shape[0]*shape[1],768)
        # TODO 搞清楚 这个learnable 参数怎么添加到这个向量当中，并且怎么拼接到一起。